## Example of using Dmatrix api to find walking distance between two connected stops. 

In [1]:
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

from datetime import timedelta
from datetime import datetime
import scipy.stats as stats

import requests as r
import pandas as pd
import seaborn as s
import numpy as np

import holidays
ie_holidays = holidays.Ireland()

import postgres
import gmaps
import googlemaps
import json
import math
import config

import xgboost

from tqdm import tnrange, tqdm_notebook

In [ ]:
data = postgres.query("SELECT * FROM combined;", tunnel=True)
data = pd.DataFrame(data)

In [4]:
stops = pd.read_csv("stop_information.csv")

cols = list(stops.columns)
cols[0] = 'ix'
stops.columns = cols
stops.drop(columns=cols[0], inplace=True)

stops.head()

,id,stopid,stop_name,lat,lng,irish_name,routes
0,4240,2,Parnell Square,53.352241,-6.263695,Cearnóg Parnell,"['38', '38A', '38B', '38D', '46A', '46E']"
1,4241,3,Parnell Square,53.352307,-6.263783,Cearnóg Parnell,"['120', '122']"
2,4242,4,Parnell Square,53.352567,-6.264166,Cearnóg Parnell,"['7', '7A', '7B', '7D', '9']"
3,4243,6,Parnell Square,53.352744,-6.264443,Cearnóg Parnell,"['4', '155']"
4,4244,7,Parnell Square,53.352836,-6.264562,Cearnóg Parnell,"['13', '140', '40', '40B', '40D']"


In [5]:
# Function to get the distance between two stops. 
def get_distance(start, finish):
    """
    Distance between two (lat,lng) pairs
    
    Inputs:
    ================================
    (int) start: stopid of first stop
    (int) finish: stopid of last stop
    
    Outputs:
    ===============================
    (int) the distance in metres between the stops. 
    
    Notes:
    ===============================
    If there is an error, or the api fails to find the distance a value of None will be returned. 
    """
    try:
        begin = (stops[stops.stopid==start ]['lat'].values[0], stops[stops.stopid==start ]['lng'].values[0])
        end   = (stops[stops.stopid==finish]['lat'].values[0], stops[stops.stopid==finish]['lng'].values[0])
    except Exception as e:
        print(start, finish)
        print(repr(e)) 
        return None
        
    API_key = config.dmatrix_key #enter Google Maps API key
    gmaps = googlemaps.Client(key=API_key)
    
    try:
        call = gmaps.distance_matrix(begin, end, mode='walking')
    
    except Exception as eL:
        print(repr(eL))
        return None
    
    status = call['status']
    
    if status=='OK':
        return call["rows"][0]["elements"][0]['distance']['value']
    
    else:
        print(status)
        return None

In [6]:
get_distance(2,3)

9

In [20]:
sample['rows']

[{'elements': [{'distance': {'text': '9 m', 'value': 9},
    'duration': {'text': '1 min', 'value': 7},
    'status': 'OK'}]}]

In [19]:
sample["rows"][0]["elements"][0]['distance']['value']

9

In [22]:
sample['status']

'OK'